In [2]:
# We need to create a SQLite database
# I'm going to use a API, but you can use CSVs

# Load data into a PANDAS dataframe -> load into the database

# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [3]:
# Fetch the data
url = "https://api.spacexdata.com/v4/launchpads"

response = requests.get(url)
data = response.json()

# create dataframe
df = pd.DataFrame(data)
df.head()

,images,name,full_name,locality,region,latitude,longitude,launch_attempts,launch_successes,rockets,timezone,launches,status,details,id
0,{'large': ['https://i.imgur.com/7uXe1Kv.png']},VAFB SLC 3W,Vandenberg Space Force Base Space Launch Compl...,Vandenberg Space Force Base,California,34.644090,-120.593144,0,0,[5e9d0d95eda69955f709d1eb],America/Los_Angeles,[],retired,SpaceX's original west coast launch pad for Fa...,5e9e4501f5090910d4566f83
1,{'large': ['https://i.imgur.com/9oEMXwa.png']},CCSFS SLC 40,Cape Canaveral Space Force Station Space Launc...,Cape Canaveral,Florida,28.561857,-80.577366,99,97,[5e9d0d95eda69973a809d1ec],America/New_York,"[5eb87cddffd86e000604b32f, 5eb87cdeffd86e00060...",active,"SpaceX's primary Falcon 9 pad, where all east ...",5e9e4501f509094ba4566f84
2,{'large': ['https://i.imgur.com/ZzTTC5p.png']},STLS,SpaceX South Texas Launch Site,Boca Chica Village,Texas,25.997264,-97.156085,0,0,[],America/Chicago,[],under construction,SpaceX's new private launch site currently und...,5e9e4502f5090927f8566f85
3,{'large': ['https://i.imgur.com/GGPgsVs.png']},Kwajalein Atoll,Kwajalein Atoll Omelek Island,Omelek Island,Marshall Islands,9.047721,167.743129,5,2,[5e9d0d95eda69955f709d1eb],Pacific/Kwajalein,"[5eb87cd9ffd86e000604b32a, 5eb87cdaffd86e00060...",retired,"SpaceX's original pad, where all of the Falcon...",5e9e4502f5090995de566f86
4,{'large': ['https://i.imgur.com/asp5L08.png']},VAFB SLC 4E,Vandenberg Space Force Base Space Launch Compl...,Vandenberg Space Force Base,California,34.632093,-120.610829,28,27,[5e9d0d95eda69973a809d1ec],America/Los_Angeles,"[5eb87ce1ffd86e000604b334, 5eb87cf0ffd86e00060...",active,SpaceX's primary west coast launch pad for pol...,5e9e4502f509092b78566f87


In [4]:
# do any data cleaning as needed
images = [x["large"][0] for x in df.images]
rockets = [len(x) for x in df.rockets]

df["images"] = images
df["rockets"] = rockets

df = df.drop(columns=["launches"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   images            6 non-null      object 
 1   name              6 non-null      object 
 2   full_name         6 non-null      object 
 3   locality          6 non-null      object 
 4   region            6 non-null      object 
 5   latitude          6 non-null      float64
 6   longitude         6 non-null      float64
 7   launch_attempts   6 non-null      int64  
 8   launch_successes  6 non-null      int64  
 9   rockets           6 non-null      int64  
 10  timezone          6 non-null      object 
 11  status            6 non-null      object 
 12  details           6 non-null      object 
 13  id                6 non-null      object 
dtypes: float64(2), int64(3), object(9)
memory usage: 800.0+ bytes


In [6]:
# Create engine using the `spacex.sqlite` database file
engine = create_engine("sqlite:///spacex.sqlite")

In [7]:
# write to the database
df.to_sql("launchpads", con=engine, index=False, method="multi", if_exists="replace")

6

In [8]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

launchpads
-----------
images TEXT
name TEXT
full_name TEXT
locality TEXT
region TEXT
latitude FLOAT
longitude FLOAT
launch_attempts BIGINT
launch_successes BIGINT
rockets BIGINT
timezone TEXT
status TEXT
details TEXT
id TEXT

